# Natural frequency of a traffic signal 🚏

In this notebook we are going to find an approximate value of the frequency of a traffic signal. We will use the Rayleigh method to reduce the continuous structure into a single degree of freedom.

## Geometry definitions

Before solving the Rayleigh integrals, some classes are defined to calculate the structural properties -linear density, point mass and stiffnes- from the geometric definition.

To avoid conversion errors, all the units are in *international system*.

In [7]:
from math import pi, sqrt

steel_density = 7700     # kg/m3
young_modulus = 210*1e9  # N/m2

class CircularSignal:
    def __init__(self, diameter, thickness):  # Input dimensions are in m
        self.mass = steel_density * thickness * pi * 0.25 * diameter**2  # kg

class SquareSignal:
    def __init__(self, width, height, thickness):  # Input dimensions are in m
        self.mass = steel_density * thickness * width * height  # kg

class CircularCrossSection:
    def __init__(self, diameter, thickness):  # Input dimensions are in m
        self.linear_density = steel_density * thickness * pi * diameter                         # kg/m
        self.stiffness = young_modulus * 1/64 * pi * (diameter**4 - (diameter-2*thickness)**4)  # N*m2

class RectangularCrossSection:
    def __init__(self, width, depth, thickness):  # Input dimensions are in m
        self.linear_density = steel_density * thickness * 2 * (width+depth)                                    # kg/m
        self.stiffness = young_modulus * 1/12 * (width*depth**3 - (width-2*thickness)*(depth-2*thickness)**3)  # N*m2

 ## Rayleigh method

The traffic signal is a structure clamped at the bottom and free at the top. Hence, a second order polynomial is choosen as shape function:
$$
\psi = \frac{z^2}{h^2} \\
\psi´´ = \frac{2}{h^2}
$$
which satisfies the kynematic boundary conditions $\psi(0)=\psi'(0)=0$.

The frequency is obtained after some algebra using the Rayleigh's expressions:
\begin{align*}
m &= \int_0^h \rho A\psi^2 dz + \sum \psi_i^2 m_i = \frac{1}{5} \rho A h + m_1 \\
k &= {\displaystyle \int_0^h EI\psi''^2 dz} = 4 \frac{EI}{h^3} \\
\end{align*}

In [8]:
def NaturalFrequencyPoly(mast_section, signal_shape, height):
    signal_shapes = signal_shape if isinstance(signal_shape, list) else [signal_shape]
    point_mass = sum([s.mass for s in signal_shapes])
    m = mast_section.linear_density * height / 5 + point_mass
    k = 4 * mast_section.stiffness / height**3
    frequency = sqrt(k/m)           # rad/s
    nat_frequency = frequency/2/pi  # Hz
    return nat_frequency

### Trigonometric shape function

$$
\psi = 1 - \cos(2\pi z/h) \\
\psi´´ = \frac{4\pi^2}{h^2}\cos(2\pi z/h)
$$

In [9]:
def NaturalFrequencyTrig(mast_section, signal_shape, height):
    signal_shapes = signal_shape if isinstance(signal_shape, list) else [signal_shape]
    point_mass = sum([s.mass for s in signal_shapes])
    m = mast_section.linear_density * height * (3/2 -4/pi) + point_mass
    k = mast_section.stiffness * pi**4 / height**3 / 32
    frequency = sqrt(k/m)           # rad/s
    nat_frequency = frequency/2/pi  # Hz
    return nat_frequency

## Calculations

In [10]:
mast = RectangularCrossSection(width=.080, depth=.040, thickness=.001)  # section dimensions are in m
signal = CircularSignal(diameter=.600, thickness=.001)                  # section dimensions are in m

nat_frequency = NaturalFrequencyPoly(mast, signal, height=3.5)   # structure dimensions are in m

print(f'Natural frequency: {nat_frequency:.1f} Hz')

Natural frequency: 3.2 Hz


In [11]:
mast = CircularCrossSection(diameter=.060, thickness=.001)       # section dimensions are in mm
signal = CircularSignal(diameter=.650, thickness=.001)           # section dimensions are in mm
signal2 = SquareSignal(width=.600, height=.800, thickness=.001)  # section dimensions are in mm

nat_frequency = NaturalFrequencyPoly(mast, [signal, signal2], height=3)    # structure dimensions are in m

print(f'Natural frequency: {nat_frequency:.1f} Hz')

Natural frequency: 3.0 Hz


In [12]:
nat_frequency = NaturalFrequencyTrig(mast, [signal, signal2], height=3)    # structure dimensions are in m

print(f'Natural frequency: {nat_frequency:.1f} Hz')

Natural frequency: 2.6 Hz
